<a href="https://colab.research.google.com/github/t-abs/Organ-Donation-Effeciency-System/blob/main/Organ_Donor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import numpy as np
!pip install faker

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 6.8 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
from faker import Faker


fake = Faker()

num_donors = 1000
num_recipients = 1000
num_matches = 1500


blood_types = ['A+', 'A-', 'B+', 'B-', 'AB+', 'AB-', 'O+', 'O-']
organ_types = ['Kidney', 'Liver', 'Heart', 'Lung', 'Pancreas']


donors = []
for i in range(num_donors):
    donors.append({
        'donor_id': i+1,
        'age': fake.random_int(min=18, max=65),
        'gender': fake.random_element(elements=('Male', 'Female', 'Other')),
        'blood_type': fake.random_element(elements=blood_types),
        'organ_type': fake.random_element(elements=organ_types),
        'height': fake.random_int(min=150, max=200),
        'weight': fake.random_int(min=50, max=120),
        'donor_city': fake.city(),
        'donation_date': fake.date_this_decade()
    })

donors_df = pd.DataFrame(donors)

# Generate recipient data
recipients = []
for i in range(num_recipients):
    recipients.append({
        'recipient_id': i+1,
        'age': fake.random_int(min=1, max=80),
        'gender': fake.random_element(elements=('Male', 'Female', 'Other')),
        'blood_type': fake.random_element(elements=blood_types),
        'organ_needed': fake.random_element(elements=organ_types),
        'height': fake.random_int(min=150, max=200),
        'weight': fake.random_int(min=50, max=120),
        'recipient_city': fake.city(),
        'waiting_time_days': fake.random_int(min=1, max=3650)
    })

recipients_df = pd.DataFrame(recipients)

# Generate match data
matches = []
for i in range(num_matches):
    donor_id = fake.random_int(min=1, max=num_donors)
    recipient_id = fake.random_int(min=1, max=num_recipients)
    match_success = int(donors_df.loc[donor_id-1, 'organ_type'] == recipients_df.loc[recipient_id-1, 'organ_needed'] and
                        donors_df.loc[donor_id-1, 'blood_type'] == recipients_df.loc[recipient_id-1, 'blood_type'])

    matches.append({
        'match_id': i+1,
        'donor_id': donor_id,
        'recipient_id': recipient_id,
        'match_success': match_success,
        'match_date': fake.date_this_decade()
    })

matches_df = pd.DataFrame(matches)

# Save the datasets to CSV files
donors_df.to_csv('donors.csv', index=False)
recipients_df.to_csv('recipients.csv', index=False)
matches_df.to_csv('matches.csv', index=False)

print("Datasets generated successfully!")


Datasets generated successfully!


In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report


donors_df = pd.read_csv('donors.csv')
recipients_df = pd.read_csv('recipients.csv')
matches_df = pd.read_csv('matches.csv')


matches_df = matches_df.merge(donors_df, on='donor_id', suffixes=('_donor', '_recipient'))
matches_df = matches_df.merge(recipients_df, on='recipient_id', suffixes=('_donor', '_recipient'))
X = matches_df[['age_donor', 'gender_donor', 'blood_type_donor', 'organ_type', 'height_donor', 'weight_donor',
                'age_recipient', 'gender_recipient', 'blood_type_recipient', 'organ_needed', 'height_recipient',
                'weight_recipient', 'waiting_time_days']]

y = matches_df['match_success']

labelencoder = LabelEncoder()
X['gender_donor'] = labelencoder.fit_transform(X['gender_donor'])
X['blood_type_donor'] = labelencoder.fit_transform(X['blood_type_donor'])
X['gender_recipient'] = labelencoder.fit_transform(X['gender_recipient'])
X['blood_type_recipient'] = labelencoder.fit_transform(X['blood_type_recipient'])
X['organ_type'] = labelencoder.fit_transform(X['organ_type'])
X['organ_needed'] = labelencoder.fit_transform(X['organ_needed'])


X = X.apply(pd.to_numeric, errors='coerce')
X.fillna(X.mean(), inplace=True)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


<ipython-input-3-9bf0a616635d>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['gender_donor'] = labelencoder.fit_transform(X['gender_donor'])
<ipython-input-3-9bf0a616635d>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['blood_type_donor'] = labelencoder.fit_transform(X['blood_type_donor'])
<ipython-input-3-9bf0a616635d>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the doc

In [4]:

model = RandomForestClassifier(n_estimators=100, random_state=42)


model.fit(X_train, y_train)


RandomForestClassifier(random_state=42)

In [5]:

y_pred = model.predict(X_test)


accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

conf_matrix = confusion_matrix(y_test, y_pred)
print(f'Confusion Matrix:\n{conf_matrix}')

class_report = classification_report(y_test, y_pred)
print(f'Classification Report:\n{class_report}')


Accuracy: 0.98
Confusion Matrix:
[[294   0]
 [  6   0]]
Classification Report:
              precision    recall  f1-score   support

           0       0.98      1.00      0.99       294
           1       0.00      0.00      0.00         6

    accuracy                           0.98       300
   macro avg       0.49      0.50      0.49       300
weighted avg       0.96      0.98      0.97       300



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [6]:

new_data = pd.DataFrame({
    'age_donor': [30, 50],
    'gender_donor': [1, 0],
    'blood_type_donor': [2, 3],
    'organ_type': [1, 0],
    'height_donor': [175, 160],
    'weight_donor': [70, 60],
    'age_recipient': [40, 55],
    'gender_recipient': [1, 0],
    'blood_type_recipient': [2, 3],
    'organ_needed': [1, 0],
    'height_recipient': [170, 165],
    'weight_recipient': [65, 70],
    'waiting_time_days': [100, 200]
})


predictions = model.predict(new_data)
print(f'Predictions: {predictions}')


Predictions: [0 0]
